In [ ]:
from __future__ import annotations

from pathlib import Path
from typing import Any

import matplotlib.pyplot as plt

from example_models import get_linear_chain_2v


def print_annotated(description: str, value: Any) -> None:
    print(
        description,
        value,
        sep="\n",
        end="\n\n",
    )

# Model building basics

In the following you will learn how to build and simulate your first model using `mxlpy`.  

This will allow you to create time courses and do steady-state analysis as shown below.

<div>
    <img src="assets/time-course.png" 
         style="vertical-align:middle; max-height: 175px; max-width: 20%;" />
    <img src="assets/protocol-time-course.png" 
         style="vertical-align:middle; max-height: 175px; max-width: 20%;" />
    <img src="assets/steady-state.png" 
         style="vertical-align:middle; max-height: 175px; max-width: 20%;" />
</div>


## Defining your first model

Let's say you want to model the following chemical network of a linear chain of reactions

$$ \Large \varnothing \xrightarrow{v_0} S \xrightarrow{v_1} P \xrightarrow{v_2} \varnothing $$

We can translate this into a system of ordinary differential equations (ODEs)

$$\begin{align*}
\frac{dS}{dt} &= v_0 - v_1     \\
\frac{dP}{dt} &= v_1 - v_2 \\
\end{align*}
$$

Note that the rates $v$ effect the variables by certain factors, known as **stoichiometries**.  
We can explicity write out these factors like this:

$$\begin{align*}
\frac{dS}{dt} &= 1 \cdot v_0 -1 \cdot v_1     \\
\frac{dP}{dt} &= 1\cdot v_1 -1 \cdot v_2 \\
\end{align*}
$$

In the example the stoichiometries are all $1$ or $-1$, however, they can have any real value.  
We can write out the stoichiometries using a **stoichiometric matrix**:

| Variable | $v_0$ | $v_1$ | $v_2$ |
| -------- | ----- | ----- | ----- |
| S        | 1     | -1    | 0     |
| P        | 0     |     1 | -1    |

Which we can read as (ignoring the 0 entries):

- `S` is produced by $v_0$ and consumed by $v_1$
- `P` is produced by $v_1$ and consumed by $v_2$ 

Lastly we choose rate equations for each rate to get the flux vector $v$

$$\begin{align*}
    v_0 &= k_{in} \\
    v_1 &= k_1 * S \\
    v_2 &= k_{out} * P \\
\end{align*}$$



## Implementing your first model

Now let's implement this first model in *mxlpy*.  
We start by creating the rate functions $\textbf{v}$.  
Note that these should be **general** and **re-usable** whenever possible, to make your model clear to people reading it.  
Try to give these functions names that are meaningful to your audience, e.g. a rate function `k * s` could be named **proportional** or **mass-action**.

In [ ]:
def constant(k: float) -> float:
    return k


def proportional(k: float, s: float) -> float:
    return k * s

Next, we create our model.  

For this, we first import the `Model` class from the `mxlpy` package.


In [ ]:
from mxlpy import Model

model = Model()

We first add parameters to the model using `.add_parameters({name: value})`.  

> Note that the function returns our `Model` object again.  
> This will be useful later, as we can *chain* multiple calls together.  

In [ ]:
from mxlpy import Parameter, units

model = model.add_parameters(
    {
        "k_in": 1,
        "k_1": Parameter(1, unit=units.mmol_s),
        "k_out": 1,
    }
)

Next we add the dynamic variables `S` and `P` with their respective initial condition.

In [ ]:
model = model.add_variables({"S": 0, "P": 0})

Finally, we add the three reactions by using 

```python
.add_reaction(
    name,              # the internal name for the reaction
    fn=...,            # a python function to be evaluated
    args=[name, ...]   # the arguments passed to the python function
    stoichiometry={    # a mapping encoding how much the variable `name`
        name: value    # is changed by the reaction
    },
)
```

> **Attention**  
> There are a couple of points to note here.  
> First, the function passed to `fn` here (and elsewhere) needs to be pickle-able  
> Thus, **lambda** functions are not supported!  
> 
> Second, the arguments defined with `args` are passed to `fn` **by position**, not by name.  
> Thus, the *order* of arguments in `args` needs to match the order of arguments in `fn`  

In [ ]:
model.add_reaction(
    "v0",
    fn=constant,
    args=["k_in"],
    stoichiometry={"S": 1},  # produces one S
)
model.add_reaction(
    "v1",
    fn=proportional,
    args=["k_1", "S"],  # note that the order needs to match `proportional`
    stoichiometry={"S": -1, "P": 1},  # consumes one S and produces one P
)
model.add_reaction(
    "v2",
    fn=proportional,
    args=["k_out", "P"],  # note that the order needs to match `proportional`
    stoichiometry={"P": -1},  # exports one P
)

print(model.get_reaction_names())

Note, that we in general recommend to use a single function that returns the model instead of defining it globally.  
This allows us to quickly re-create the model whenever we need a fresh version of it.  
Below, we define the same model again, but inside a single function.  


> Note that we made use of **operator chaining** to avoid having to write `model` for every call.  

So we can write `Model.method1().method2()...`  instead of having to write

```python
model.method1()
model.method2()
```

etc

In [ ]:
from mxlpy import Variable


def create_linear_chain_model() -> Model:
    return (
        Model()
        .add_parameters(
            {
                "k_in": Parameter(1, unit=units.mmol_s),
                "k_1": Parameter(1, unit=units.per_second),
                "k_out": Parameter(1, unit=units.per_second),
            }
        )
        .add_variables(
            {
                "S": Variable(0, unit=units.mmol),
                "P": Variable(0, unit=units.mmol),
            }
        )
        .add_reaction(
            "v0",
            fn=constant,
            args=["k_in"],
            stoichiometry={"S": 1},  # produces one S
            unit=units.mmol_s,
        )
        .add_reaction(
            "v1",
            fn=proportional,
            args=["k_1", "S"],  # note that the order needs to match `proportional`
            stoichiometry={"S": -1, "P": 1},  # consumes one S and produces one P
            unit=units.mmol_s,
        )
        .add_reaction(
            "v2",
            fn=proportional,
            args=["k_out", "P"],  # note that the order needs to match `proportional`
            stoichiometry={"P": -1},  # exports one P
            unit=units.mmol_s,
        )
    )

We can then simulate the model by passing it to a `Simulator` and simulate a time series using `.simulate(t_end)`.  
Finally, we can obtain the concentrations and fluxes using `get_result`.  

While you can directly plot the `pd.DataFrame`s, mxlpy supplies a variety of plots in the `plot` namespace that are worth checking out.  

In [ ]:
from mxlpy import Simulator, plot

res = (
    Simulator(create_linear_chain_model())  # initialise the simulator
    .simulate(5)  # simulate until t_end = 5 a.u.
    .get_result()
    .value
)

if isinstance(res, Exception):
    raise res
else:
    variables, fluxes = res

    fig, (ax1, ax2) = plot.two_axes(figsize=(6, 2.5))
    _ = plot.lines(variables, ax=ax1)
    _ = plot.lines(fluxes, ax=ax2)

    # Never forget to labelr you axes :)
    ax1.set(xlabel="time / a.u.", ylabel="concentration / a.u.")
    ax2.set(xlabel="time / a.u.", ylabel="flux / a.u.")
    plt.show()

Note, that we explicitly checked whether the simulation failed.  

Since there can be a multitude of reasons why a simulation might fail, we return a `Result[Simulation  | Exception]` object, which you can use to investigate potential errors.

In contrast, if you are **sure** the simulation won't fail, and still want your code to be type-safe, you can use `.unwrap_or_err()`.

```python
result = Simulator(model).simulate(10).get_result().unwrap_or_err()
```

Note that these functions will raise an error if the simulation failed, which potentially might crash your programs.

## Derived quantities

Frequently it makes sense to derive one quantity in a model from other quantities.  
This can be done for

- parameters derived from other parameters
- variables derived from parameters or other variables
- stoichiometries derived from parameters or variables (more on this later)

*mxlpy* offers a unified interface for derived parameters and variables usign `Model.add_derived()`.  

In [ ]:
def moiety_1(x1: float, total: float) -> float:
    return total - x1


def model_derived() -> Model:
    return (
        Model()
        .add_variables({"ATP": 1.0})
        .add_parameters({"ATP_total": 1.0, "k_base": 1.0, "e0_atpase": 1.0})
        .add_derived("k_atp", proportional, args=["k_base", "e0_atpase"])
        .add_derived("ADP", moiety_1, args=["ATP", "ATP_total"])
        .add_reaction(
            "ATPase", proportional, args=["k_atp", "ATP"], stoichiometry={"ATP": -1}
        )
    )


variables, fluxes = Simulator(model_derived()).simulate(10).get_result().unwrap_or_err()
fig, ax = plot.lines(variables)
ax.set(xlabel="time / a.u.", ylabel="concentration / a.u.")
plt.show()

## Introspection

If the simulation didn't show the expected results, it is usually a good idea to try to pinpoint the error.  
`mxlpy` offers a variety of methods to access intermediate results.  

The first is to check whether all **derived quantities** were calculate correctly.  
For this, you can use the `get_args` method, which is named consistently with the `args` argument in all methods like `add_reaction`.

In [ ]:
m = create_linear_chain_model()

print_annotated(
    "Using initial conditions as default:",
    m.get_args(),
)

print_annotated(
    "Using custom concentrations:",
    m.get_args({"S": 1.0, "P": 0.5}),
)

If the `args` look fine, the next step is usually to check whether the rate equations are looking as expected

In [ ]:
m = create_linear_chain_model()
print_annotated(
    "Using initial conditions as default:",
    m.get_fluxes(),
)
print_annotated(
    "Using custom concentrations:",
    m.get_fluxes({"S": 1.0, "P": 0.5}),
)

and whether the stoichiometries are assigned correctly

In [ ]:
m = create_linear_chain_model()
m.get_stoichiometries()

Lastly, you can check the generated right hand side

In [ ]:
m = create_linear_chain_model()

print_annotated(
    "Using initial conditions as default:",
    m.get_right_hand_side(),
)

print_annotated(
    "Using custom concentrations:",
    m.get_right_hand_side({"S": 1.0, "P": 0.5}),
)

If any of the quantities above were unexpected, you can check the model interactively by accessing the various collections.  

> Note: the returned quantities are **copies** of the internal data, modifying these won't have any effect on the model

In [ ]:
m.parameters

In [ ]:
m.variables

In [ ]:
m.reactions

In case you model contains derived quantitites you can access the derived quantities using `.derived`.  
Note that this returns a **copy** of the derived quantities, so editing it won't have any effect on the model.  

In [ ]:
model_derived().derived

## CRUD

The model has a complete **c**reate, **r**ead, **u**pdate, **d**elete API for all it's elements.  
The methods and attributes are named consistenly, with `add` instead of `create` and `get` instead of `read`.  
Note that the elements itself are accessible as `properties`, e.g. `.parameters` which will return **copies** of the data.  
Only use the supplied methods to change the internal state of the model.

Here are some example methods and attributes for parameters

| Functionality | Parameters                                                                              |
| ------------- | --------------------------------------------------------------------------------------- |
| Create        | `.add_parameter()`, `.add_parameters()`                                                 |
| Read          | `.parameters`, `.get_parameter_names()`                                                 |
| Update        | `.update_parameter()`, `.update_parameters()`, `.scale_parameter()`, `scale.parameters()` |
| Delete        | `.remove_parameter()`, `.remove_parameters()`                                           |

and variables

| Functionality | Variables                                                         |
| ------------- | ----------------------------------------------------------------- |
| Create        | `.add_variable()`, `.add_variables()`                             |
| Read          | `.variables`, `.get_variable_names()`, `get_initial_conditions()` |
| Update        | `.update_variable()`, `.update_variables()`                         |
| Delete        | `.remove_parameter()`, `.remove_parameters()`                     |


In [ ]:
m = create_linear_chain_model()

# Calculate fluxes
print_annotated(
    "Before update",
    m.get_fluxes({"S": 1.0, "P": 0.5}),
)

# Update parameters
m.update_parameters({"k_in": 2.0})

# Calculate fluxes again
print_annotated(
    "After update",
    m.get_fluxes({"S": 1.0, "P": 0.5}),
)

## Derived stoichiometries

To define derived stoichiometries can make them dependent on parameters in the model or use the `Derived` class as a value in the stoichiometries.  

So instead of defining them like this

`stoichiometry={"x": 1.0}`

you can use

`stoichiometry={"x": "stoich"}`

or for more advanced uses you use the `Derived` class as the value

`stoichiometry={"x": Derived(fn=constant, args=["stoich"])}`

In [ ]:
variables, fluxes = (
    Simulator(
        Model()
        .add_parameters({"stoich": -1.0, "k": 1.0})
        .add_variables({"x": 1.0})
        .add_reaction(
            "name",
            proportional,
            args=["x", "k"],
            # Define derived stoichiometry here
            stoichiometry={"x": "stoich"},
        )
    )
    .simulate(1)
    # Update parameter the derived stoichiometry depends on
    .update_parameter("stoich", -4.0)
    # Continue simulation
    .simulate(5)
    .get_result()
    .unwrap_or_err()
)

_, ax = plot.lines(variables)
ax.set(xlabel="time / a.u.", ylabel="concentration / a.u.")
plt.show()

## Simulations: time courses

Time courses are simulations over time

<img src="assets/time-course.png" style="max-width: 500px" />

You can obtain the time course of integration using the `simulate` method.  
There are two ways how you can define the time points this function returns.  

1. supply the end time `t_end`
2. supply both end time and number of steps with `t_end` and `steps`

If you want to set the exact time points to be returned use `simulate_time_course`

```python
simulate(t_end=10)
simulate(t_end=10, steps=10)
simulate_time_course(np.linspace(0, 10, 11))
```

In [ ]:
variables, fluxes = (
    Simulator(get_linear_chain_2v())
    .simulate(t_end=10)  # simulate until t_end = 10 a.u.
    .get_result()
    .unwrap_or_err()
)

fig, ax = plot.lines(variables)
ax.set(xlabel="time / a.u.", ylabel="concentration / a.u.")
plt.show()


By default, the `Simulator` is initialised with the initial concentrations set in the `Model`.  
Optionally, you can overwrite the initial conditions using the `y0` argument.  

```python
Simulator(model, y0={name: value, ...})
```

In [ ]:
variables, fluxes = (
    Simulator(create_linear_chain_model(), y0={"S": 2.0, "P": 0.0})
    .simulate(10)
    .get_result()
    .unwrap_or_err()
)

fig, (ax1, ax2) = plot.two_axes(figsize=(6, 3))
_ = plot.lines(variables, ax=ax1)
_ = plot.lines(fluxes, ax=ax2)

ax1.set(xlabel="time / a.u.", ylabel="concentration / a.u.")
ax2.set(xlabel="time / a.u.", ylabel="flux / a.u.")
plt.show()

### Inspecting the result object

The object returned by the simulator has a couple of neat features

In [ ]:
res = (
    Simulator(create_linear_chain_model(), y0={"S": 2.0, "P": 0.0})
    .simulate(10)
    .get_result()
    .unwrap_or_err()
)

You can unpack the results object into variables and fluxes

In [ ]:
concs, fluxes = res

And you can get reactions that either produce or consume a certain variable (that is, their stoichiometries are positive or negative).

You can **scale** these fluxes by their respective stoichiometry using `scaled=True` as well.


In [ ]:
res.get_producers("S").head()

In [ ]:
res.get_consumers("S").head()

You can get the right hand side of the simulation for every time step

In [ ]:
res.get_right_hand_side().head()

## Simulations: protocol time course

Protocols are used to make parameter changes discrete in time, such as turning a light on and off.  
This is useful reproducing experimental time courses where a parameter was changed at fixed time points.  


<img src="assets/protocol-time-course.png" style="max-width: 500px" />

The protocol is defined as a `pandas.DataFrame` using `pd.Timedelta` values as in index, and the parameter values at the respective time interval as values.  

|    pd.Timedelta  | p1 | p2 |
| ---------------- | -- | -- |
| 0 days 00:00:01  |  1 |  0 |
| 0 days 00:00:03  |  2 |  1 |
| 0 days 00:00:06  |  1 |  2 |

You can use as many parameters as you want.  

> **Note**  
> *mxlpy* assigns one second of the `Timedelta` to one **time unit** of the integration.  
> mxlpy does **not** take into account whether your integration might use a different time unit.  

For convenience, we supply the `make_protocol` function, which takes in a pair of the **duration** of the time-step on the respective **parameter values**.  

In [ ]:
from mxlpy import make_protocol

protocol = make_protocol(
    [
        (1, {"k1": 1}),  # for one second value of 1
        (2, {"k1": 2}),  # for two seconds value of 2
        (3, {"k1": 1}),  # for three seconds value of 1
    ]
)
protocol

Now instead of running `simulate` or `simulate_time_course` we use `simulate_protocol` or `simulate_protocol_time_course`

In [ ]:
variables, fluxes = (
    Simulator(get_linear_chain_2v())
    .simulate_protocol(protocol)  # changed here
    .get_result()
    .unwrap_or_err()
)

fig, ax = plot.one_axes()
plot.lines(variables, ax=ax)
plot.shade_protocol(protocol["k1"], ax=ax, alpha=0.1)
ax.set(xlabel="time / a.u.", ylabel="concentration / a.u.")
plt.show()

In [ ]:
import numpy as np

variables, fluxes = (
    Simulator(get_linear_chain_2v())
    .simulate_protocol_time_course(
        protocol,
        time_points=np.linspace(0, 6, 101, dtype=float),
    )
    .get_result()
    .unwrap_or_err()
)

fig, ax = plt.subplots()
plot.lines(variables, ax=ax)
plot.shade_protocol(protocol["k1"], ax=ax, alpha=0.1)
ax.set(xlabel="time / a.u.", ylabel="concentration / a.u.")
plt.show()

## Simulations: steady-state

A steady-state describes a state at which the concentrations of the system don't change anymore (also called fixed points).

<img src="assets/steady-state.png" style="max-width: 500px" />

You can simulate until the model reaches a steady-state using the `simulate_to_steady_state` method.  


In [ ]:
variables, fluxes = (
    Simulator(get_linear_chain_2v())  # optionally supply initial conditions
    .simulate_to_steady_state()
    .get_result()
    .unwrap_or_err()
)

fig, ax = plot.bars(variables)
ax.set(xlabel="Variable / a.u.", ylabel="Concentration / a.u.")
plt.show()

## SBML

The systems biology markup language (SBML) is a widely used file format for sharing models between different software packages and programming languages.  

`mxlpy` supports reading and writing **sbml** models using the `sbml.read` and `sbml.write` functions.

In [ ]:
from mxlpy import sbml

model = sbml.read(Path("assets") / "00001-sbml-l3v2.xml")
variables, fluxes = Simulator(model).simulate(10).get_result().unwrap_or_err()
_ = plot.lines(variables)

When exporting a model, you can supply additional meta-information like units and compartmentalisation.  
See the [official sbml documentation](https://sbml.org/documents/) for more information of legal values.

In [ ]:
sbml.write(
    model,
    file=Path(".cache") / "model.xml",
    extent_units="mole",
    substance_units="mole",
    time_units="second",
)

<div style="color: #ffffff; background-color: #04AA6D; padding: 3rem 1rem 3rem 1rem; box-sizing: border-box">
    <h2>First finish line</h2>
    With that you now know most of what you will need from a day-to-day basis about model building and simulation in mxlpy.
    <br />
    <br />
    Congratulations!
</div>

## Advanced topics

### Time-dependent reactions

You can use the special name `time` to refer to the actual integration time in the rare case a reaction or module depends on it explicitly.  
This is why the methods `get_args`, `get_fluxes` etc. also take an additional `time` argument.  

In [ ]:
def time_dependency() -> Model:
    return (
        Model()
        .add_variable("x", 1.0)
        .add_reaction(
            "v1",
            proportional,
            args=["time", "x"],
            stoichiometry={"x": -1},
        )
    )


model = time_dependency()

# Watch our for explicit time dependency here!
print_annotated(
    "Fluxes at time = 1.0",
    model.get_fluxes(time=1.0),
)
print_annotated(
    "Fluxes at time = 2.0",
    model.get_fluxes(time=2.0),
)

# During simulations the time is automatically taken care of
_ = (
    Simulator(model)
    .simulate(t_end=10)
    .get_result()
    .unwrap_or_err()
    .variables.plot(
        xlabel="time / a.u.",
        ylabel="amount / a.u.",
        title="Time-dependent reaction",
    )
)

### Derived parameters and variables

Internally mxlpy differentiates between derived *parameters* and derived *variables*.  
This differentiation is just-in-time before any calculation and thus might change if you change the nature of a parameter / variable.  

If you are interested in which category mxlpy has placed the derived quantities, you can access `.derived_parameters` and `.derived_variables` as well. 

In [ ]:
def model_derived() -> Model:
    return (
        Model()
        .add_variables({"ATP": 1.0})
        .add_parameters({"ATP_total": 1.0, "k_base": 1.0, "e0_atpase": 1.0})
        .add_derived("k_atp", proportional, args=["k_base", "e0_atpase"])
        .add_derived("ADP", moiety_1, args=["ATP", "ATP_total"])
        .add_reaction(
            "ATPase", proportional, args=["k_atp", "ATP"], stoichiometry={"ATP": -1}
        )
    )


m = Model().add_parameters({"x1": 1.0}).add_derived("x1d", constant, args=["x1"])
print("Derived Parameters:", m.get_derived_parameters())
print("Derived Variables:", m.get_derived_variables())

print("\nMaking x1 dynamic")
m.make_parameter_dynamic("x1")
print("Derived Parameters:", m.get_derived_parameters())
print("Derived Variables:", m.get_derived_variables())

## Derived initial conditions

MxlPy supports initial assignments. These are essentially derived values which are just calculated **once**.  

In [ ]:
from mxlpy import fns
from mxlpy.types import InitialAssignment

(
    Model()
    .add_parameters(
        {
            "k1": 0.1,
            "k2": InitialAssignment(fn=fns.twice, args=["k1"]),
        }
    )
    .add_variables(
        {
            "v1": 0.1,
            "v2": InitialAssignment(fn=fns.proportional, args=["k2", "v1"]),
        }
    )
).get_args()

## Data

You can add references to data using `add_data`.  
That way, you can, for example, dynamically derive aggregates over them. 

In [ ]:
import pandas as pd


def average(light: pd.Series) -> float:
    return light.mean()


def model_data(light: pd.Series) -> Model:
    return (
        Model()
        .add_data("light", light)
        .add_derived("averge_light", average, args=["light"])
    )


lights = pd.Series(
    data={"400nm": 200, "500nm": 300, "600nm": 400},
    dtype=float,
)

model_data(lights).get_args()